[Reference](https://medium.com/geekculture/visualize-the-german-railway-system-with-kepler-gl-91aea7f063a9)

In [1]:
!pip install keplergl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 16.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached jedi-0.19.0-py2.py3-none-any.whl (1.6 MB)
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922317 sha256=64a9fe85ffaa1477f61fce8207920eeb95e5eef211d638687b740537ab3e56ec
  Stored in directory: /root/.cache/pip/wheels/e5/e9/db/b8b8f2fcb5cb1cd7268d88959fce267bc00697cb8f50dcd0d9
Successfully built keplergl


In [ ]:
from keplergl import KeplerGl
import pandas as pd
import requests
from google.colab import output
from datetime import datetime
output.enable_custom_widget_manager()
import os
from getpass import getpass
Client_Id = getpass('Enter DB API Client_Id: ')
Api_Key = getpass('Enter DB API Api_Key: ')

if not os.path.exists("data"):
    os.mkdir("data")

bahnhof = pd.read_csv("https://raw.githubusercontent.com/dgg32/db_kepler/master/data/bahnhof.csv")
bahnhof.head()

bahnhof[bahnhof["NAME"] == "Hamburg Hbf"]["DS100"].values[0]

def search_route(start, end, time, time_type):
    #curl -X POST -H "Accept: application/json" -H "Content-Type: application/json"
    #"https://openapi.trassenfinder.de/3.12/api/v5/routen/suche"
    # -d '{"infrastruktur_id": 10, "sucheinstellungen": {"an_abzeit": "2022-11-13T16:38:36+02:00", "verkehrsart": "spfv_lok", "zeitvorgabe_typ": "abzeit"},
    # "wegpunkte": [{"betriebsstelle": {"ds100": "HBS"}}, {"betriebsstelle": {"ds100":"MH"}}]}'
    start_ds100 = bahnhof[bahnhof["NAME"] == start]["DS100"].values[0]
    end_ds100 = bahnhof[bahnhof["NAME"] == end]["DS100"].values[0]

    headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}
    data = {"infrastruktur_id": 10, "sucheinstellungen": {"an_abzeit": time, "verkehrsart": "spfv_lok", "zeitvorgabe_typ": time_type},
    "wegpunkte": [{"betriebsstelle": {"ds100": start_ds100}}, {"betriebsstelle": {"ds100":end_ds100}}]}

    #print (data)

    r = requests.post('https://openapi.trassenfinder.de/3.12/api/v5/routen/suche', headers=headers, json=data)

    return r.json()

route = search_route("Braunschweig Hbf", "München Hbf", "2022-11-09T07:16:36+02:00", "abzeit")
route["result"]["gewichtete_route"]["routenpunkte"][:5]

source_lat = -1000
source_lng = -1000

target_lat = -1000
target_lng = -1000

previous_station = ""

cumulative_time = 0

cumulative_energy = 0

cumulative_distance = 0

content = ",".join(["source_lat", "source_lng", "target_lat", "target_lng", "from_to", "cumulative_distance_km", "cumulative_energy_kwh", "cumulative_min"]) + "\n"

#route["result"]["gewichtete_route"]["routenpunkte"]
for s in route["result"]["gewichtete_route"]["routenpunkte"]:
    ds100 = s["ds100"]
    #print (s["technische_fahrzeit_info"])
    if source_lat == -1000:
        try:
            source_lat = bahnhof[bahnhof["DS100"] == ds100]["latitude"].values[0]
            source_lng = bahnhof[bahnhof["DS100"] == ds100]["longitude"].values[0]
            previous_station = bahnhof[bahnhof["DS100"] == ds100]["NAME"].values[0]
            #distance = s["laufende_hm"] - cumulative_distance
            cumulative_distance = s["laufende_hm"]/10

            #energy = s["energieverbrauch_info"]["energieverbrauch_gesamt_kwh"] - cumulative_energy
            cumulative_energy = s["energieverbrauch_info"]["energieverbrauch_gesamt_kwh"]

            cumulative_time = s["technische_fahrzeit_info"]["ankunft_min"]

            content += f"{source_lat},{source_lng},{source_lat},{source_lng},{previous_station},{cumulative_distance},{cumulative_energy},{cumulative_time}\n"

        except:
            pass
    else:
        try:


            target_lat = bahnhof[bahnhof["DS100"] == ds100]["latitude"].values[0]
            target_lng = bahnhof[bahnhof["DS100"] == ds100]["longitude"].values[0]


            #distance = s["laufende_hm"] - cumulative_distance
            cumulative_distance = s["laufende_hm"]/10

            #energy = s["energieverbrauch_info"]["energieverbrauch_gesamt_kwh"] - cumulative_energy
            cumulative_energy = s["energieverbrauch_info"]["energieverbrauch_gesamt_kwh"]

            cumulative_time = s["technische_fahrzeit_info"]["ankunft_min"]
            current_station = bahnhof[bahnhof["DS100"] == ds100]["NAME"].values[0]

            from_to = f"{previous_station} -> {current_station}"
            previous_station = current_station

            content += f"{source_lat},{source_lng},{target_lat},{target_lng},{from_to},{cumulative_distance},{cumulative_energy},{cumulative_time}\n"

            source_lat = target_lat
            source_lng = target_lng
        except:
            pass


with open("data/data_route.csv", "w") as output:
    output.write(content)

route_df = pd.read_csv("data/data_route.csv")
route_df.head()


route_df["between_distance_km"] = route_df["cumulative_distance_km"].diff().round(2)
route_df["between_energy_kwh"] = route_df["cumulative_energy_kwh"].diff()
route_df["between_min"] = route_df["cumulative_min"].diff()
route_df["between_distance_km"].fillna(0, inplace=True)
route_df["between_energy_kwh"].fillna(0, inplace=True)
route_df["between_min"].fillna(0, inplace=True)

route_df.head()

config = {'version': 'v1',
 'config': {'visState': {
   'layers': [{
     'type': 'arc',
     'config': {'dataId': 'route',
      'label': 'source -> target arc',
      'color': [201, 0, 0],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat0': 'source_lat',
       'lng0': 'source_lng',
       'lat1': 'target_lat',
       'lng1': 'target_lng'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'thickness': 6.7,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'sizeRange': [0, 10],
       'targetColor': [155, 165, 46]},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}},
    {
     'type': 'point',
     'config': {'dataId': 'bahnhof',
      'label': 'Point',
      'color': [18, 92, 119],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat': 'latitude', 'lng': 'longitude', 'altitude': None},
      'isVisible': True,
      'visConfig': {'radius': 10,
       'fixedRadius': False,
       'opacity': 0.09,
       'outline': False,
       'thickness': 2,
       'strokeColor': None,
       'colorRange': {'name': 'Uber Viz Qualitative 4',
        'type': 'qualitative',
        'category': 'Uber',
        'colors': ['#12939A',
         '#DDB27C',
         '#88572C',
         '#FF991F',
         '#F15C17',
         '#223F9A',
         '#DA70BF',
         '#125C77',
         '#4DC19C',
         '#776E57',
         '#17B8BE',
         '#F6D18A',
         '#B7885E',
         '#FFCB99',
         '#F89570',
         '#829AE3',
         '#E79FD5',
         '#1E96BE',
         '#89DAC1',
         '#B3AD9E']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radiusRange': [0, 50],
       'filled': True},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': {'name': 'state', 'type': 'string'},
      'colorScale': 'ordinal',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}},
    {
     'type': 'point',
     'config': {'dataId': 'route',
      'label': 'target',
      'color': [19, 164, 171],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat': 'target_lat', 'lng': 'target_lng', 'altitude': None},
      'isVisible': True,
      'visConfig': {'radius': 21.1,
       'fixedRadius': False,
       'opacity': 0.91,
       'outline': False,
       'thickness': 2,
       'strokeColor': None,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radiusRange': [0, 50],
       'filled': True},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'bahnhof': [{'name': 'EVA_NR',
        'format': None},
       {'name': 'DS100', 'format': None},
       {'name': 'NAME', 'format': None},
       {'name': 'Verkehr', 'format': None},
       {'name': 'state', 'format': None}],
      'route': [{'name': 'from_to', 'format': None},
       {'name': 'cumulative_distance_km', 'format': None},
       {'name': 'cumulative_energy_kwh', 'format': None},
       {'name': 'cumulative_min', 'format': None},
       {'name': 'between_distance_km', 'format': None},
       {'name': 'between_energy_kwh', 'format': None},
       {'name': 'between_min', 'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}}
}}

map_1 = KeplerGl(height=800, data={"bahnhof": bahnhof, "route": route_df}, config=config)
from IPython.display import Javascript
display(Javascript('''
  google.colab.widgets.installCustomManager('https://ssl.gstatic.com/colaboratory-static/widgets/colab-cdn-widget-manager/6a14374f468a145a/manager.min.js');
'''))

map_1
map_1.config

target_location = "Berlin Hbf"
travel_date = "2022-11-07"

def search_location(location):

    headers = {
    'DB-Client-Id': Client_Id,
    'DB-Api-Key': Api_Key,
    'accept': "application/json"
    }

    #print (data)

    r = requests.get(f'https://apis.deutschebahn.com/db-api-marketplace/apis/fahrplan/v1/location/{location}', headers=headers)

    r_json = r.json()

    for s in r_json:
        if s["name"] == location:
            return s

    #return r.json()

location = search_location(target_location)
location

def journey_detail(detailsId, from_station):
    headers = {
    'DB-Client-Id': Client_Id,
    'DB-Api-Key': Api_Key,
    'accept': "application/json"
    }


    r = requests.get(f'https://apis.deutschebahn.com/db-api-marketplace/apis/fahrplan/v1/journeyDetails/{detailsId}', headers=headers)
    #r.encoding = 'utf-8'
    r_json = r.json()
    print (r_json)
    stations = {}
    begin_recording = False
    for t in r_json:
        #print (detailsId)
        #print (t)
        if from_station in t["stopName"]:

            begin_recording = True

        if begin_recording == True:
            if "&#x0028;" in  t["stopName"]:
                 t["stopName"] = t["stopName"].replace("&#x0028;", '(')
            if "&#x0029;" in t["stopName"]:
                 t["stopName"] = t["stopName"].replace("&#x0029;", ')')
            #print (t)
            arrTime = None
            depTime = None

            if "depTime" in t:
                depTime = t["depTime"]

            if "arrTime" in t:
                arrTime = t["arrTime"]
            stations[t["stopId"]] = {"lat": float(t["lat"]), "lon": float(t["lon"]), "stopName": t["stopName"], "train": t["train"],  "type": t["type"], "arrTime": arrTime, "depTime": depTime}

    return stations

def depart_from(location_id, date):
    headers = {
    'DB-Client-Id': Client_Id,
    'DB-Api-Key': Api_Key,
    'accept': "application/json"
    }

    #print (data)

    trains = {}
    for h in range(24):
        #print (f"{date}T{str(h).zfill(2)}:00")
        r = requests.get(f'https://apis.deutschebahn.com/db-api-marketplace/apis/fahrplan/v1/departureBoard/{location_id}?date={date}T{str(h).zfill(2)}:00', headers=headers)

        r_json = r.json()
        #print (r_json)
        for t in r_json:
            train_date = datetime.strptime(t["dateTime"], '%Y-%m-%dT%H:%M').date()
            my_date = datetime.strptime(date, '%Y-%m-%d').date()
            #print (t)
            if train_date == my_date:
                trains[t["detailsId"]] = {"name": t["name"], "type": t["type"], "direction": t["direction"], "dateTime": t["dateTime"], "boardId": t["boardId"]}

    return trains

def arrive_at(location_id, date):
    headers = {
    'DB-Client-Id': Client_Id,
    'DB-Api-Key': Api_Key,
    'accept': "application/json"
    }

    #print (data)

    trains = {}
    for h in range(24):
        #print (f"{date}T{str(h).zfill(2)}:00")
        r = requests.get(f'https://apis.deutschebahn.com/db-api-marketplace/apis/fahrplan/v1/arrivalBoard/{location_id}?date={date}T{str(h).zfill(2)}:00', headers=headers)

        r_json = r.json()
        #print (r_json)
        for t in r_json:
            train_date = datetime.strptime(t["dateTime"], '%Y-%m-%dT%H:%M').date()
            my_date = datetime.strptime(date, '%Y-%m-%d').date()
            #print (t)
            if train_date == my_date:
                trains[t["detailsId"]] = {"name": t["name"], "type": t["type"], "origin": t["origin"], "dateTime": t["dateTime"], "boardId": t["boardId"]}

    return trains

journey = arrive_at(location["id"], travel_date)
journey